In [2]:
import mamba_ssm
print(mamba_ssm.__version__)

/root/miniconda/envs/RingTool/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.2.4


In [3]:
import torch
print(torch.cuda.is_available())  # 检查 CUDA 是否可用
print(torch.version.cuda)  # 查看 PyTorch 支持的 CUDA 版本

True
11.8


In [4]:
import torch
print(torch.cuda.is_available())  # 如果输出 True，说明 CUDA 可用
print(torch.cuda.device_count())  # 输出可用的 GPU 数量


True
4


In [4]:
import os
import json
import random

# 定义数据路径和可用受试者
data_path = "/root/RingTool/data/rings/"
available_subjects = ['00005', '00009', '00012', '00020', '00022', '00029', '00031']
config_dir = "/root/RingTool/config/supervised/"

# 确保配置目录存在
if not os.path.exists(config_dir):
    print(f"Config directory {config_dir} does not exist!")
    exit(1)

# 遍历 supervised 文件夹下的所有 JSON 文件
config_files = [os.path.join(root, f) for root, _, files in os.walk(config_dir) for f in files if f.endswith(".json")]

print(f"Found {len(config_files)} JSON config files in {config_dir}")

# 为 5 折交叉验证生成划分
random.seed(42)  # 设置随机种子以确保可重现
random.shuffle(available_subjects)  # 随机打乱受试者列表

# 手动分配 7 个受试者到 5 折，确保无空折
five_fold = {
    "Fold-1": available_subjects[0:2],  # ['00005', '00009']
    "Fold-2": available_subjects[2:4],  # ['00012', '00020']
    "Fold-3": available_subjects[4:5],  # ['00022']
    "Fold-4": available_subjects[5:6],  # ['00029']
    "Fold-5": available_subjects[6:7]   # ['00031']
}

# 遍历每个 JSON 文件并更新 split 配置
for config_path in config_files:
    try:
        # 加载配置
        with open(config_path, "r") as f:
            config = json.load(f)

        # 更新 split
        config["split"]["train"] = available_subjects[:5]  # ['00005', '00009', '00012', '00020', '00022']
        config["split"]["valid"] = available_subjects[5:6]  # ['00029']
        config["split"]["test"] = available_subjects[6:]   # ['00031']
        config["split"]["5-Fold"] = five_fold

        # 保存更新后的配置文件
        with open(config_path, "w") as f:
            json.dump(config, f, indent=4)

        # 打印更新信息
        print(f"\nUpdated config: {config_path}")
        print(f"Split: train={config['split']['train']}, valid={config['split']['valid']}, test={config['split']['test']}")
        print(f"5-Fold: {config['split']['5-Fold']}")
        print(f"Input type: {config['dataset']['input_type']}")
        print(f"Batch size: {config['dataset']['batch_size']}")
        print(f"Device: {config['train']['device']}")

    except Exception as e:
        print(f"Error processing {config_path}: {str(e)}")

print("\nAll configurations have been updated!")

Found 33 JSON config files in /root/RingTool/config/supervised/

Updated config: /root/RingTool/config/supervised/ring2/hr/ir/mamba2-ring2-hr-all-ir.json
Split: train=['00009', '00020', '00022', '00012', '00031'], valid=['00005'], test=['00029']
5-Fold: {'Fold-1': ['00009', '00020'], 'Fold-2': ['00022', '00012'], 'Fold-3': ['00031'], 'Fold-4': ['00005'], 'Fold-5': ['00029']}
Input type: ['ir-filtered']
Batch size: 32
Device: 0

Updated config: /root/RingTool/config/supervised/ring2/hr/ir/transformer-ring2-hr-all-ir.json
Split: train=['00009', '00020', '00022', '00012', '00031'], valid=['00005'], test=['00029']
5-Fold: {'Fold-1': ['00009', '00020'], 'Fold-2': ['00022', '00012'], 'Fold-3': ['00031'], 'Fold-4': ['00005'], 'Fold-5': ['00029']}
Input type: ['ir-filtered']
Batch size: 128
Device: 0

Updated config: /root/RingTool/config/supervised/ring2/hr/ir/resnet-ring2-hr-all-ir.json
Split: train=['00009', '00020', '00022', '00012', '00031'], valid=['00005'], test=['00029']
5-Fold: {'Fold

In [ ]:
! python /root/RingTool/main.py --data-path /root/RingTool/data/rings --config config/only_test/resnet-ring1-hr-all-ir-paper.json

In [ ]:
python /root/RingTool/main.py --data-path /root/RingTool/data/rings --batch-configs-dirs config/physical-based

In [ ]:
nohup python3 main.py --data-path /root/RingTool/data/rings --batch-configs-dirs config/only_test > $(date +\%Y\%m\%d\%H\%M\%S)-nohup-test.log 2>&1 &

In [5]:
import os
import json
import random

# 定义数据路径和可用受试者
data_path = "/root/RingTool/config/only_test/"
available_subjects = ['00005', '00009', '00012', '00020', '00022', '00029', '00031']
config_dir = "/root/RingTool/config//"

# 确保配置目录存在
if not os.path.exists(config_dir):
    print(f"Config directory {config_dir} does not exist!")
    exit(1)

# 遍历 supervised 文件夹下的所有 JSON 文件
config_files = [os.path.join(root, f) for root, _, files in os.walk(config_dir) for f in files if f.endswith(".json")]

print(f"Found {len(config_files)} JSON config files in {config_dir}")

# 为 5 折交叉验证生成划分
random.seed(42)  # 设置随机种子以确保可重现
random.shuffle(available_subjects)  # 随机打乱受试者列表

# 手动分配 7 个受试者到 5 折，确保无空折
five_fold = {
    "Fold-1": available_subjects[0:2],  # ['00005', '00009']
    "Fold-2": available_subjects[2:4],  # ['00012', '00020']
    "Fold-3": available_subjects[4:5],  # ['00022']
    "Fold-4": available_subjects[5:6],  # ['00029']
    "Fold-5": available_subjects[6:7]   # ['00031']
}

# 遍历每个 JSON 文件并更新 split 配置
for config_path in config_files:
    try:
        # 加载配置
        with open(config_path, "r") as f:
            config = json.load(f)

        # 更新 split
        config["split"]["train"] = available_subjects[:5]  # ['00005', '00009', '00012', '00020', '00022']
        config["split"]["valid"] = available_subjects[5:6]  # ['00029']
        config["split"]["test"] = available_subjects[6:]   # ['00031']
        config["split"]["5-Fold"] = five_fold

        # 保存更新后的配置文件
        with open(config_path, "w") as f:
            json.dump(config, f, indent=4)

        # 打印更新信息
        print(f"\nUpdated config: {config_path}")
        print(f"Split: train={config['split']['train']}, valid={config['split']['valid']}, test={config['split']['test']}")
        print(f"5-Fold: {config['split']['5-Fold']}")
        print(f"Input type: {config['dataset']['input_type']}")
        print(f"Batch size: {config['dataset']['batch_size']}")
        print(f"Device: {config['train']['device']}")

    except Exception as e:
        print(f"Error processing {config_path}: {str(e)}")

print("\nAll configurations have been updated!")

Found 160 JSON config files in /root/RingTool/config//

Updated config: /root/RingTool/config//only_test/resnet-ring1-hr-all-ir-paper.json
Split: train=['00009', '00020', '00022', '00012', '00031'], valid=['00005'], test=['00029']
5-Fold: {'Fold-1': ['00009', '00020'], 'Fold-2': ['00022', '00012'], 'Fold-3': ['00031'], 'Fold-4': ['00005'], 'Fold-5': ['00029']}
Input type: ['ir-filtered']
Batch size: 128
Device: 0

Updated config: /root/RingTool/config//only_test/ring2/hr/ir/stationary-scenes/inception-time-ring2-hr-stationary-ir.json
Split: train=['00009', '00020', '00022', '00012', '00031'], valid=['00005'], test=['00029']
5-Fold: {'Fold-1': ['00009', '00020'], 'Fold-2': ['00022', '00012'], 'Fold-3': ['00031'], 'Fold-4': ['00005'], 'Fold-5': ['00029']}
Input type: ['ir-filtered']
Batch size: 128
Device: 0

Updated config: /root/RingTool/config//only_test/ring2/hr/ir/stationary-scenes/transformer-ring2-hr-stationary-ir.json
Split: train=['00009', '00020', '00022', '00012', '00031'], va

In [ ]:
! nohup python3 main.py --data-path /root/RingTool/data/rings --batch-configs-dirs config/supervised > $(date +\%Y\%m\%d\%H\%M\%S)-nohup-train.log 2>&1 &